# Optuna study

> Combine it with papermill and wandb for seamless hyperparameter tuning

In [1]:
import os
import optuna
from tsai.optuna import *
import papermill as pm
from tsai.optuna import run_optuna_study
from fastcore.basics import *
from optuna.distributions import *
from optuna.samplers import TPESampler
import wandb

/usr/local/pip-global/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = AttrDict(
    study_name = 'general_study_extended', # name of the Optuna study
    study_type = 'bayesian', # 'bayesian' or 'gridsearch' or 'random'
    n_trials = 50, # number of trials
    train_nb = f'{os.getcwd()}/solfsmy_train.ipynb', # path to the notebook to be executed
    search_space = {
        "arch.attn_dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1),
        "arch.d_model": IntUniformDistribution(32, 512, 32),
        "arch.d_ff": IntUniformDistribution(32, 512, 32),
        "arch.decomposition": CategoricalDistribution([True, False]),
        "arch.dropout": DiscreteUniformDistribution(0.0, 0.5, 0.1), 
        "arch.individual": CategoricalDistribution([True, False]),
        "arch.n_layers": IntUniformDistribution(1, 6, 1),
        "arch.n_heads": CategoricalDistribution([2, 4, 8, 16, 32]),
        "init_weights": CategoricalDistribution([True, False]), # true = kaiming
        "lookback": CategoricalDistribution([18, 24, 36, 128, 192])  # Change here
    },
    # Add extra parameters that are fixed, but not part of the search space
    extra_params = {
        "n_epoch": 50,
        "bs": 128
    },
    use_wandb = True,
    wandb_mode = 'offline'
)

config

```json
{ 'extra_params': {'bs': 128, 'n_epoch': 50},
  'n_trials': 50,
  'search_space': { 'arch.attn_dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1),
                    'arch.d_ff': IntUniformDistribution(high=512, low=32, step=32),
                    'arch.d_model': IntUniformDistribution(high=512, low=32, step=32),
                    'arch.decomposition': CategoricalDistribution(choices=(True, False)),
                    'arch.dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1),
                    'arch.individual': CategoricalDistribution(choices=(True, False)),
                    'arch.n_heads': CategoricalDistribution(choices=(2, 4, 8, 16, 32)),
                    'arch.n_layers': IntUniformDistribution(high=6, low=1, step=1),
                    'init_weights': CategoricalDistribution(choices=(True, False)),
                    'lookback': CategoricalDistribution(choices=(18, 24, 36, 128, 192))},
  'study_name': 'general_study_extended',
  'study_type': 'bayesian',
  'train_nb': '/workspaces/sw-driver-forecaster/dev_nbs/solfsmy_train.ipynb',
  'use_wandb': True,
  'wandb_mode': 'offline'}
```

In [3]:
def create_objective(train_nb, search_space, extra_params=None, use_wandb=False):
    """
        Create objective function to be minimized by Optuna.
        Inputs:
            trial: Optuna trial object
            train_nb: path to the training notebook
            search_vars: keys of the search space to be used
            wandb_group: name of the wandb group to be used
        Output:
            valid_loss: validation loss
    """
    def objective(trial:optuna.Trial):
        # Define the parameters to be passed to the training notebook through papermill
        pm_parameters = {}
        for k,v in search_space.items():
            pm_parameters['config.' + k] = trial._suggest(k, v)

        # Add the extra parameters to the dictionary. The key of every parameter 
        # must be 'config.<param_name>'
        if extra_params is not None:
            for k,v in extra_params.items():
                pm_parameters['config.' + k] = v
                
        # If using wandb, enable that in the training runs, all of them gathered
        # into a group (NOTE: The train nb must have and use these config arguments)
        if use_wandb:
            pm_parameters['config.use_wandb'] = True
            pm_parameters['config.wandb_group'] = config.study_name + '_runs'

        # Call the training notebook using papermill (don't print the output)
        stdout_file = open('tmp/pm_stdout.txt', 'w')
        stderr_file = open('tmp/pm_stderr.txt', 'w')

        pm.execute_notebook(
            train_nb,
            './tmp/pm_output.ipynb',
            parameters = pm_parameters,
            stdout_file = stdout_file,
            stderr_file = stderr_file
        )

        # Close the output files
        stdout_file.close()
        stderr_file.close()

        # Get the output value of interest from the source notebook
        loss = None
        %store -r valid_loss
        print(f'Validation loss: {valid_loss}')
        return valid_loss

    return objective

In [4]:
obj = create_objective(config.train_nb, config.search_space, 
                       extra_params=config.extra_params, use_wandb=True)
study = run_optuna_study(obj, study_type='bayesian', direction='minimize', path='./tmp',
                 study_name=config.study_name, n_trials=config.n_trials)

[I 2024-07-08 10:26:11,904] A new study created in memory with name: general_study_extended
  0%|          | 0/50 [00:00<?, ?it/s]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group
Best trial: 0. Best value: 313.616:   2%|▏         | 1/50 [00:28<23:24, 28.67s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown 

Validation loss: 313.6161804199219
[I 2024-07-08 10:26:40,574] Trial 0 finished with value: 313.6161804199219 and parameters: {'arch.attn_dropout': 0.4, 'arch.d_model': 320, 'arch.d_ff': 224, 'arch.decomposition': False, 'arch.dropout': 0.0, 'arch.individual': False, 'arch.n_layers': 3, 'arch.n_heads': 2, 'init_weights': True, 'lookback': 128}. Best is trial 0 with value: 313.6161804199219.


Best trial: 0. Best value: 313.616:   4%|▍         | 2/50 [00:54<21:31, 26.92s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


Validation loss: 357.7586364746094
[I 2024-07-08 10:27:06,264] Trial 1 finished with value: 357.7586364746094 and parameters: {'arch.attn_dropout': 0.1, 'arch.d_model': 256, 'arch.d_ff': 64, 'arch.decomposition': False, 'arch.dropout': 0.5, 'arch.individual': True, 'arch.n_layers': 4, 'arch.n_heads': 32, 'init_weights': True, 'lookback': 18}. Best is trial 0 with value: 313.6161804199219.


Best trial: 2. Best value: 150.979:   6%|▌         | 3/50 [01:19<20:32, 26.22s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


...

Validation loss: 213.92738342285156
[I 2024-07-08 10:50:48,156] Trial 47 finished with value: 213.92738342285156 and parameters: {'arch.attn_dropout': 0.1, 'arch.d_model': 96, 'arch.d_ff': 64, 'arch.decomposition': False, 'arch.dropout': 0.2, 'arch.individual': True, 'arch.n_layers': 4, 'arch.n_heads': 16, 'init_weights': False, 'lookback': 192}. Best is trial 28 with value: 129.62025451660156.


Best trial: 28. Best value: 129.62:  98%|█████████▊| 49/50 [25:06<00:30, 30.56s/it]Passed unknown parameter: config.arch.attn_dropout
Passed unknown parameter: config.arch.d_model
Passed unknown parameter: config.arch.d_ff
Passed unknown parameter: config.arch.decomposition
Passed unknown parameter: config.arch.dropout
Passed unknown parameter: config.arch.individual
Passed unknown parameter: config.arch.n_layers
Passed unknown parameter: config.arch.n_heads
Passed unknown parameter: config.init_weights
Passed unknown parameter: config.lookback
Passed unknown parameter: config.n_epoch
Passed unknown parameter: config.bs
Passed unknown parameter: config.use_wandb
Passed unknown parameter: config.wandb_group


Validation loss: 129.7626495361328
[I 2024-07-08 10:51:18,364] Trial 48 finished with value: 129.7626495361328 and parameters: {'arch.attn_dropout': 0.5, 'arch.d_model': 128, 'arch.d_ff': 32, 'arch.decomposition': True, 'arch.dropout': 0.0, 'arch.individual': True, 'arch.n_layers': 5, 'arch.n_heads': 16, 'init_weights': False, 'lookback': 24}. Best is trial 28 with value: 129.62025451660156.


Best trial: 28. Best value: 129.62: 100%|██████████| 50/50 [25:35<00:00, 30.70s/it]

Validation loss: 135.56272888183594
[I 2024-07-08 10:51:46,934] Trial 49 finished with value: 135.56272888183594 and parameters: {'arch.attn_dropout': 0.5, 'arch.d_model': 32, 'arch.d_ff': 192, 'arch.decomposition': True, 'arch.dropout': 0.1, 'arch.individual': False, 'arch.n_layers': 6, 'arch.n_heads': 32, 'init_weights': False, 'lookback': 24}. Best is trial 28 with value: 129.62025451660156.

Optuna study saved to tmp/general_study_extended.pkl
To reload the study run: study = joblib.load('tmp/general_study_extended.pkl')

Study statistics    : 
  Study name        : general_study_extended
  # finished trials : 50
  # pruned trials   : 0
  # complete trials : 50

Best trial          :
  value             : 129.62025451660156
  best_params = {'arch.attn_dropout': 0.2, 'arch.d_model': 64, 'arch.d_ff': 192, 'arch.decomposition': True, 'arch.dropout': 0.0, 'arch.individual': True, 'arch.n_layers': 5, 'arch.n_heads': 8, 'init_weights': False, 'lookback': 24}



In [5]:
run = wandb.init(config=config, mode=config.wandb_mode, 
                 job_type='optuna-study') if config.use_wandb else None

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [9]:
!sudo pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 5.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.9 MB/s eta 0:00:00


In [10]:
if run is not None:
    run.log(dict(study.best_params, **{'best_value': study.best_value, 
                                       'best_trial_number': study.best_trial.number}))
    run.log_artifact(f'./tmp/{config.study_name}.pkl', type='optuna_study')
    run.log({
        'contour': optuna.visualization.plot_contour(study),
        'edf': optuna.visualization.plot_edf(study),
        'intermediate_values': optuna.visualization.plot_intermediate_values(study),
        'optimization_history': optuna.visualization.plot_optimization_history(study),
        'parallel_coordinate' : optuna.visualization.plot_parallel_coordinate(study),
        'param_importances': optuna.visualization.plot_param_importances(study),
        'slice': optuna.visualization.plot_slice(study)
    })

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [ ]:
if run is not None:
    run.finish()